# model overfitting solution
  - Dropout 적용
  - image 증식

In [1]:
from tensorflow.keras import Sequential # keras model 
from tensorflow.keras.layers import Conv2D, MaxPool2D # Convolution layer
from tensorflow.keras.layers import Dense, Dropout, Flatten # Affine layer
import os

# dir setting
base_dir = "C:/IITT/6_Tensorflow/workspace/chap07_CNN/lecture_3_Cats_and_Dogs/cats_and_dogs"

train_dir = os.path.join(base_dir, 'train_dir')
validation_dir = os.path.join(base_dir, 'validation_dir')

# Hyper parameters
img_h = 150 # height
img_w = 150 # width
input_shape = (img_h, img_w, 3) 

# 1. CNN Model layer 
print('model create')
model = Sequential()

# Convolution layer1 : kernel[3,3,3,32]
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu',
                 input_shape = input_shape))
model.add(MaxPool2D(pool_size=(2,2)))

# Convolution layer2 : kernel[3,3,32,64]
model.add(Conv2D(64,kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

# Convolution layer3 : kernel[5,5,64,128], maxpooling() 제외 
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

# Flatten layer :4d -> 2d
model.add(Flatten()) 
# 2차 적용 : 드롭아웃 - 과적합 해결 
model.add(Dropout(0.5)) # fully connected 층 이전에 배치 

# Affine layer(Fully connected layer1) : [n, 256]
model.add(Dense(256, activation = 'relu'))

# Output layer(Fully connected layer2) : [256, 1]
model.add(Dense(1, activation = 'sigmoid'))

# model training set : Adam or RMSprop 
model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy', # one hot encoding
              metrics = ['accuracy'])

# 2. image file preprocessing : 이미지 제너레이터 이용  
from tensorflow.keras.preprocessing.image import ImageDataGenerator
print("image preprocessing")

# 1차 적용 
#train_data = ImageDataGenerator(rescale=1./255)

# 2차 적용 : image 증식 - 과적합 해결
train_data = ImageDataGenerator(
        rescale=1./255,
        rotation_range = 40, # image 회전 각도 범위(+, - 범위)
        width_shift_range = 0.2, # image 수평 이동 범위
        height_shift_range = 0.2, # image 수직 이용 범위  
        shear_range = 0.2, # image 전단 각도 범위
        zoom_range=0.2, # image 확대 범위
        horizontal_flip=True,) # image 수평 뒤집기 범위 

# 검증 데이터에는 증식 적용 안함 
validation_data = ImageDataGenerator(rescale=1./255)

train_generator = train_data.flow_from_directory(
        train_dir,
        target_size=(150,150),
        batch_size=35, 
        class_mode='binary') # binary label
# Found 2000 images belonging to 2 classes.

validation_generator = validation_data.flow_from_directory(
        validation_dir,
        target_size=(150,150),
        batch_size=35, # [수정] batch size 올림 
        class_mode='binary')
# Found 1000 images belonging to 2 classes.

model create
image preprocessing
Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [2]:

# 3. model training : 배치 제너레이터 이용 모델 훈련 
model_fit = model.fit_generator(
          train_generator, 
          steps_per_epoch=100, 
          epochs=10, # 30 epochs()
          validation_data=validation_generator,
          validation_steps=50) 

Epoch 1/10
100/100 [==============================] - 81s 814ms/step - loss: 0.7158 - accuracy: 0.5113 - val_loss: 0.6881 - val_accuracy: 0.5510
Epoch 2/10
100/100 [==============================] - 66s 659ms/step - loss: 0.6855 - accuracy: 0.5478 - val_loss: 0.6715 - val_accuracy: 0.5464
Epoch 3/10
100/100 [==============================] - 65s 649ms/step - loss: 0.6589 - accuracy: 0.5950 - val_loss: 0.6557 - val_accuracy: 0.6352
Epoch 4/10
100/100 [==============================] - 67s 673ms/step - loss: 0.6631 - accuracy: 0.5899 - val_loss: 0.6373 - val_accuracy: 0.6294
Epoch 5/10
100/100 [==============================] - 67s 675ms/step - loss: 0.6436 - accuracy: 0.6141 - val_loss: 0.6200 - val_accuracy: 0.6663
Epoch 6/10
100/100 [==============================] - 67s 673ms/step - loss: 0.6128 - accuracy: 0.6584 - val_loss: 0.5540 - val_accuracy: 0.7193
Epoch 7/10
100/100 [==============================] - 67s 675ms/step - loss: 0.5969 - accuracy: 0.6781 - val_loss: 0.5692 - val_ac